In [1]:
import sys, logging

def setup_logging(level=logging.INFO, to_stdout=True, include_name=True):
    fmt = "[%(asctime)s] %(levelname)s: "
    if include_name:
        fmt += "%(name)s: "
    fmt += "%(message)s"

    handler = logging.StreamHandler(sys.stdout if to_stdout else sys.stderr)
    handler.setFormatter(logging.Formatter(fmt))

    root = logging.getLogger()
    root.handlers.clear()
    root.setLevel(level)
    root.addHandler(handler)

    logging.getLogger("transformers").setLevel(logging.WARNING)
    logging.getLogger("qdrant_client").setLevel(logging.INFO)
    logging.getLogger("httpx").setLevel(logging.WARNING)

logger = logging.getLogger(__name__) 

In [2]:
import os
import time
import json
import logging
from typing import List, Dict, Any, Optional

import torch
import numpy as np
from PIL import Image
from pdf2image import convert_from_path
from torch.utils.data import DataLoader
from tqdm import tqdm

from transformers.utils.import_utils import is_flash_attn_2_available
from colpali_engine.models import ColQwen2_5, ColQwen2_5_Processor
from transformers import Qwen2_5_VLForConditionalGeneration, Qwen2_5_VLProcessor
from qwen_vl_utils import process_vision_info

from qdrant_client import QdrantClient
from qdrant_client.models import (
    Distance, VectorParams, PointStruct,
    MultiVectorConfig, MultiVectorComparator
)

def log_env():
    cuda = torch.cuda.is_available()
    logger.info(f"CUDA available: {cuda}")
    if cuda:
        logger.info(f"GPU name: {torch.cuda.get_device_name(0)}")
    logger.info(f"FlashAttention2 available: {is_flash_attn_2_available()}")

# ---------- Qdrant----------
class QdrantVectorStore:
    def __init__(
        self,
        url: str = "http://qdrant:6333",
        api_key: Optional[str] = None,
        collection_name: str = "colpali_documents",
        timeout: float = 120.0,
        prefer_grpc: bool = True,
        grpc_port: int = 6334,
    ):
        """
        QdrantVectorStore の初期化

        Args:
            url (str): Qdrant サーバーのURL (デフォルト: http://qdrant:6333)
            api_key (Optional[str]): APIキー
            collection_name (str): コレクション名。RDBのテーブルのような概念
            timeout (float): タイムアウト秒数
            prefer_grpc (bool): RESTよりgRPCを優先して通信する（バイナリデータの転送で、高速・低レイテンシ）
            grpc_port (int): gRPCポート番号 (デフォルト: 6334)
        """
        self.collection_name = collection_name

        #QdrantClientのインスタンス化
        self.client = QdrantClient(
            url=url,
            api_key=api_key,
            timeout=timeout,
            prefer_grpc=prefer_grpc,
            grpc_port=grpc_port,
        )
        logger.info(f"Qdrant client initialized with URL: {url}")

    def create_collection(
        self,
        multivector_size: int = 256,
        force_recreate: bool = True,
        distance: Distance = Distance.COSINE,
        comparator: MultiVectorComparator = MultiVectorComparator.MAX_SIM,
    ) -> None:
        """
        Qdrantにコレクションを作成するメソッド（マルチベクター対応）

        Args:
            multivector_size: 埋め込みモデルが出力するベクトルの次元数
            force_recreate: Trueの場合、既存コレクションを削除して再作成
            Distance.COSINE: 検索にコサイン類似度を使用
            comparator(MultiVectorComparator.MAX_SIM): 複数のベクトルの中で一番スコアが高い(類似度が高い)ものを選ぶ

        Raises:
            Exception: コレクション作成時にエラーが発生した場合
        """
        try:
            exists = any(
                col.name == self.collection_name
                for col in self.client.get_collections().collections
            )
            if exists:
                if force_recreate:
                    logger.info(f"Deleting existing collection: {self.collection_name}")
                    self.client.delete_collection(self.collection_name)
                else:
                    logger.info(f"Collection {self.collection_name} already exists")
                    return
                    
            #コレクション作成メソッドの呼び出し
            self.client.create_collection(
                collection_name=self.collection_name,
                vectors_config=VectorParams(
                    size=multivector_size,
                    distance=distance,
                    multivector_config=MultiVectorConfig(comparator=comparator),
                ),
            )
            logger.info(
                f"Created collection {self.collection_name} with multivector support"
            )
        except Exception as e:
            logger.error(f"Error creating collection: {e}")
            raise

    def _to_multivector(self, embedding: Any) -> List[List[float]]:
        """
        Qdrantにマルチベクターを登録する前処理
        Embeddingの形状が1次元でも2次元でも、2次元リストに正規化する
        """
        arr = (
            #embeddingがTensorならCPUに移動し、fp32に変換したうえでNumPy配列にする
            #Tensorでない場合、NumPy配列に変換し、精度をfp32に統一
            embedding.detach().cpu().to(torch.float32).numpy()
            if isinstance(embedding, torch.Tensor)
            else np.array(embedding, dtype=np.float32)
        )
        #1次元のベクトルの場合[0.1, 0.2]、 2次元リストにラップして返す [[0.1, 0.2]]
        if arr.ndim == 1:
            return [arr.tolist()]
        #2次元ベクトルの場合、そのままのpythonリストに変換して返す
        if arr.ndim == 2:
            return arr.tolist() 
        raise ValueError(f"Unexpected embedding shape: {arr.shape}")

    def store_embeddings(
        self,
        embeddings: List[torch.Tensor],
        metadata: List[Dict[str, Any]],
        upsert_batch_size: int = 64,
        wait: bool = False,
    ) -> None:
        """
        Qdrantへembeddingをアップサート
        """
        #embeddingsとmetadataの数が一致しているかをチェック
        assert len(embeddings) == len(metadata), "embeddings/metadata length mismatch"

        sent = 0
        buf: List[PointStruct] = []

        def flush(_points: List[PointStruct]):
            """
            バッファに溜まったポイントをQdrantに送信する内部関数
            ポイントはQdrantの基本単位で、1件のベクトルデータを表すレコード(id,ベクトル,メタデータ)。
            バッファは複数のポイントを一時的に貯めておくリスト
            """
            if not _points:
                return
            t0 = time.perf_counter()

            # バッファに溜まったポイントをQdrantに一括送信
            self.client.upsert(
                collection_name=self.collection_name,
                points=_points,
                wait=wait, #False: WAL(ログ)への追記が終わった時点で次のバッファを送る → データファイルへの書き込み、インデックス更新はQdrant内で非同期処理
            )
            dt = time.perf_counter() - t0
            logger.info(f"Qdrant upsert {len(_points)} pts in {dt:.2f}s")

        try:
            # embeddingとmetadataを1件ずつ処理
            for i, (emb, meta) in enumerate(zip(embeddings, metadata)):
                # embeddingを2次元リストに正規化
                multivector = self._to_multivector(emb)
                # Qdrantに送信可能な形式に変換。（id, ベクトル, メタデータ)=ポイント
                pid = int(meta.get("global_page_num", i))
                buf.append(PointStruct(id=pid, vector=multivector, payload=meta))
                # バッファがbatch_sizeに達したらQdrant送信
                if len(buf) >= upsert_batch_size:
                    flush(buf)
                    sent += len(buf)
                    buf = []
            # ループ終了後に、余ったバッファがあれば送信
            if buf:
                flush(buf)
                sent += len(buf)
            logger.info(f"Stored total {sent} embeddings in Qdrant")
        except Exception as e:
            logger.error(f"Error storing embeddings: {e}")
            raise

    def search(
        self,
        query_embedding: torch.Tensor,
        top_k: int = 3,
        score_threshold: Optional[float] = None,
    ) -> List[Dict[str, Any]]:
        """
        検索クエリの埋め込みベクトルを、Qdrantが受け付ける形式（2次元リスト）に変換する。
        Qdrantで類似検索を実行して結果を整形して返す。
    
        Args:
            query_embedding: 検索クエリの埋め込みベクトル
            top_k: 上位何件を返すか（デフォルト: 3）
            score_threshold : 類似度スコアのしきい値（低スコアを除外）
        
        Notes:
            Qdrant API呼び出し時の主なパラメータ:
              - collection_name: 検索対象のコレクション名
              - query: 検索クエリベクトル（正規化済み）
              - limit: 上位件数 (top_k)
              - score_threshold: 類似度スコアのしきい値
              - with_payload=True: 結果にペイロード（メタデータ）を含める
        """
        try:
            # クエリ埋め込みをQdrantに渡せる形（2次元リスト）に正規化
            query = self._to_multivector(query_embedding)
            # Qdrantへ検索リクエストを送信
            res = self.client.query_points(
                collection_name=self.collection_name,
                query=query,
                limit=top_k,
                score_threshold=score_threshold,
                with_payload=True,
            )
            # 検索結果（id,スコア,メタデータ）を整形してPythonの辞書リストに変換
            out = [
                {"id": p.id, "score": float(p.score), "payload": p.payload}
                for p in res.points
            ]
            logger.info(f"Found {len(out)} results")
            return out
        except Exception as e:
            logger.error(f"Error during search: {e}")
            raise

    def get_collection_info(self) -> Dict[str, Any]:
        """
        Qdrantのコレクション情報を取得
    
        Returns:
            - name: コレクション名
            - vectors_count: 登録されているベクトルの総数
            - points_count: 登録されているポイントの総数
            - status: コレクションの状態
        """
        try:
            # Qdrantからコレクション情報を取得
            info = self.client.get_collection(self.collection_name)

            # 必要な情報を辞書にまとめて返す
            return {
                "name": self.collection_name,
                "vectors_count": info.vectors_count,
                "points_count": info.points_count,
                "status": info.status,
            }
        except Exception as e:
            logger.error(f"Error getting collection info: {e}")
            return {}

# ----------MultiModalRAG----------
class MultiModalRAG:
    def __init__(
        self,
        retrieval_model_name: str = "vidore/colqwen2.5-v0.2", # 検索用埋め込みモデル
        vlm_model_name: str = "Qwen/Qwen2.5-VL-32B-Instruct", # 回答生成に用いるVLMモデル
        qdrant_url: str = "http://qdrant:6333", #QdrantサーバーのURL
        qdrant_api_key: Optional[str] = None,
        collection_name: str = "colpali_documents", # Qdrantコレクション名
        use_qdrant: bool = True,
        pdf_dpi: int = 150, # PDF→画像変換時の解像度
        pdf_fmt: str = "jpeg", # PDF→画像変換時のフォーマット（jpegはI/O軽量）
        use_pdftocairo: bool = True, # pdf画像変換で、pdftocairoを利用する(高品質なレンダリング)
    ):
        """
        MultiModalRAGクラスの初期化処理。
        """
        self.retrieval_model_name = retrieval_model_name
        self.vlm_model_name = vlm_model_name
        self.use_qdrant = use_qdrant
        self.pdf_dpi = pdf_dpi
        self.pdf_fmt = pdf_fmt
        self.use_pdftocairo = use_pdftocairo

        self.retrieval_model = None
        self.retrieval_processor = None

        self.vlm_model = None
        self.vlm_processor = None

        self.vector_store = QdrantVectorStore(
            url=qdrant_url,
            api_key=qdrant_api_key,
            collection_name=collection_name,
            timeout=120.0,
            prefer_grpc=True,
            grpc_port=6334,
        )

        self.all_document_images: List[Image.Image] = [] # PDFから変換した画像リスト
        self.all_document_metadata: List[Dict[str, Any]] = [] # 各画像に対応するメタデータ
        self.all_document_embeddings: List[torch.Tensor] = [] # 各画像ベクトル
        self.document_mapping: Dict[str, Dict[str, int]] = {} # PDF名ごとのページ範囲マッピング

    # ---- モデル読み込み ----
    def load_retrieval(self) -> None:
        """
        検索用埋め込みモデル（ColQwen2.5）をロード
        """
        logger.info(f"Loading retrieval model: {self.retrieval_model_name}")
        t0 = time.perf_counter()

        # 検索用埋め込みモデルをロード（検索用）
        self.retrieval_model = ColQwen2_5.from_pretrained(
            self.retrieval_model_name,
            torch_dtype=torch.bfloat16,
            device_map="cuda:0" if torch.cuda.is_available() else "cpu",
            attn_implementation="flash_attention_2" if is_flash_attn_2_available() else None, # FlashAttention2対応なら高速化を有効にする
        ).eval()
        # プロセッサをロード（入力画像/テキストを埋め込み入力形式に整形）
        self.retrieval_processor = ColQwen2_5_Processor.from_pretrained(
            self.retrieval_model_name,
            use_fast=True
        )
        logger.info(f"Retrieval loaded in {time.perf_counter() - t0:.2f}s")

    def load_vlm(self) -> None:
        """
        回答生成に使用する視覚言語モデル（Qwen2.5-VL）とそのプロセッサをロード
        """
        logger.info(f"Loading VLM model: {self.vlm_model_name}")
        t0 = time.perf_counter()
        # VLMモデル本体をロード（生成用）
        self.vlm_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
            self.vlm_model_name,
            torch_dtype=torch.bfloat16,
            device_map="cuda:0" if torch.cuda.is_available() else "cpu",
            attn_implementation="flash_attention_2" if is_flash_attn_2_available() else None, # FlashAttention2対応なら高速化を有効にする
        ).eval()
        
        # 入力画像サイズの下限と上限を指定（ピクセル数ベース）
        min_pixels = 256 * 28 * 28
        max_pixels = 2048 * 28 * 28

        # VLM用のプロセッサをロード（画像・テキストを入力形式に整形）
        self.vlm_processor = Qwen2_5_VLProcessor.from_pretrained(
            self.vlm_model_name,
            min_pixels=min_pixels, 
            max_pixels=max_pixels,
            use_fast=True
        )
        logger.info(f"VLM loaded in {time.perf_counter() - t0:.2f}s")

    def pdf_to_images(self, pdf_path: str) -> List[Image.Image]:
        """
        PDFファイルをページごとに画像へ変換する関数

        Args:
            pdf_path (str): 入力PDFファイルのパス
            
        Returns:
            List[Image.Image]: 変換後の各ページ画像のリスト
        """
        logger.info(f"Converting PDF to images: {pdf_path} (dpi={self.pdf_dpi})")
        
         # PDFファイルの存在チェック（なければ例外を投げる）
        if not os.path.exists(pdf_path):
            raise FileNotFoundError(f"PDF file not found: {pdf_path}")
            
        t0 = time.perf_counter()

        # pdf2imageを使ってPDFを画像に変換
        images = convert_from_path(
            pdf_path,
            dpi=self.pdf_dpi, # 解像度（dpi）
            use_pdftocairo=self.use_pdftocairo, # pdftocairoを使うか（高速・正確)
            fmt=self.pdf_fmt, #出力画像フォーマット（jpeg）
        )
        dt = time.perf_counter() - t0
        logger.info(f"pdf2image({os.path.basename(pdf_path)}): {len(images)} pages in {dt:.2f}s")
        return images

    def process_multiple_pdfs(
        self,
        pdf_paths: List[str],
        batch_size: int = 16,
        num_workers: int = 8,
        prefetch_factor: int = 2,
        force_recreate_collection: bool = False,
        upsert_batch_size: int = 256,
        upsert_wait: bool = False,
    ) -> None:
        """
        複数のPDFファイルを読み込み、画像・メタデータ・ベクトルを生成して保持する
        
        Args:
            pdf_paths (List[str]): 入力PDFファイルのパスリスト
            batch_size (int): 埋め込み生成時のDataLoaderバッチサイズ
            num_workers (int): DataLoaderで並列に使うワーカー数
            prefetch_factor (int): 各ワーカーが先読みするバッチ数
            force_recreate_collection (bool): TrueならQdrantコレクションを再作成
            upsert_batch_size (int):  Qdrantへ送る“書き込みバッチ”の件数。store_embeddings() にそのまま引き継がれる。
            upsert_wait (bool): False: WAL(ログ)への追記が終わった時点でレスポンス返却 → データファイルへの書き込みとインデックス更新はQdrant内で非同期処理
        """
        if self.retrieval_model is None or self.retrieval_processor is None:
            raise ValueError("Retrieval model not loaded. Call load_retrieval() first.")

        all_images: List[Image.Image] = []
        all_metadata: List[Dict[str, Any]] = []
        start_page = 0

        for pdf_path in pdf_paths:
            logger.info(f"Processing PDF: {pdf_path}")
            # PDFファイルの存在確認（なければスキップ）
            if not os.path.exists(pdf_path):
                logger.warning(f"PDF file not found: {pdf_path}")
                continue

            # PDFをページごとに画像へ変換
            images = self.pdf_to_images(pdf_path)
            pdf_name = os.path.basename(pdf_path)

            # ページ範囲を記録（ドキュメントマッピング用）
            end_page = start_page + len(images)
            self.document_mapping[pdf_name] = {
                "start_page": start_page,
                "end_page": end_page,
                "total_pages": len(images),
            }

            # ページごとのメタデータを作成
            metadata = [
                {
                    "pdf_path": pdf_path,
                    "pdf_name": pdf_name,
                    "page_num": i + 1, # PDF内のページ番号（1始まり）
                    "global_page_num": start_page + i, # 全体での通しページ番号
                    "image_path": f"{pdf_name}_page_{i+1}.{self.pdf_fmt}", # 変換後の画像ファイル名
                }
                for i in range(len(images))
            ]

            # 全体リストに追加(複数PDF分の変換データ)
            all_images.extend(images)
            all_metadata.extend(metadata)
            start_page = end_page # 次のPDFのページ開始位置を更新
            
            logger.info(f"Added {len(images)} pages from {pdf_name}")

        # PDFから画像に変換した結果をインスタンス変数に保存（後で埋め込み生成や検索に利用）
        self.all_document_images = all_images
        self.all_document_metadata = all_metadata

        # 画像を埋め込みモデルの入力形式に変換するためのヘルパー関数
        def collate_fn(batch_imgs: List[Image.Image]):
            return self.retrieval_processor.process_images(batch_imgs)

        # DataLoaderの設定（画像を埋め込みモデルに効率的に流し込み）
        dl_kwargs = dict(
            batch_size=max(1,batch_size),
            shuffle=False, # 順序を固定（PDFのページ順を維持）
            collate_fn=collate_fn, # 画像を埋め込みモデルの入力形式に変換する前処理関数
            num_workers=num_workers, # データローダーのワーカー数
            pin_memory=torch.cuda.is_available(),
            persistent_workers=(num_workers > 0), # ワーカーを持続させる（エポックごとにワーカーを再作成しない）
        )
        if num_workers > 0:
            dl_kwargs["prefetch_factor"] = prefetch_factor # GPUに渡す次バッチを、CPUワーカーが先に用意しておく深さ。GPUを待たせないため・

        dataloader = DataLoader(all_images, **dl_kwargs)

        logger.info(
            f"Creating embeddings for {len(all_images)} pages from {len(pdf_paths)} PDFs..."
            f" (batch_size={dl_kwargs['batch_size']}, num_workers={num_workers}"
            f", prefetch_factor={prefetch_factor if num_workers>0 else 'n/a'})"
        )

        # 行列演算の高速化設定（TF32を使った最適化）
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.set_float32_matmul_precision("high")

        # 埋め込みを溜めるリスト
        embeddings: List[torch.Tensor] = []
        t0 = time.perf_counter()

        first_batch_logged = False

        # 画像のベクトル変換
        with torch.inference_mode(), torch.amp.autocast("cuda", dtype=torch.bfloat16):
            for bi, batch_images in enumerate(tqdm(dataloader, desc="Embedding pages (opt)")):
                t_start = time.perf_counter()
                # GPUへ転送
                batch_images = {
                    k: v.to(self.retrieval_model.device, non_blocking=True)
                    for k, v in batch_images.items()
                }
                t_ready = time.perf_counter()

                # 画像のベクトル変換
                out = self.retrieval_model(**batch_images)
                t_done = time.perf_counter()

                # バッチ出力を1件ずつ分解してリストに追加
                embeddings.extend(torch.unbind(out))

                data_wait = t_ready - t_start  # データ準備にかかった時間
                compute = t_done - t_ready  # 推論計算にかかった時間
                logger.info(f"[batch {bi:03d}] bs={out.shape[0]} data_wait={data_wait:.3f}s compute={compute:.3f}s")
                if not first_batch_logged:
                    logger.info(f"first_batch_total_latency={t_done - t_start:.3f}s")
                    first_batch_logged = True

        # CUDA環境ならGPUの処理が終わるまで同期
        if torch.cuda.is_available():
            torch.cuda.synchronize()

        t1 = time.perf_counter()
        total = len(embeddings)
        logger.info(f"throughput: {total / (t1 - t0):.2f} images/sec, total={total}")
        self.all_document_embeddings = embeddings
        logger.info(f"Created embeddings for {total} pages from {len(pdf_paths)} PDFs")

        # 画像ベクトルをQdrantへ保存
        if self.use_qdrant and self.vector_store and total > 0:
            emb_dim = embeddings[0].shape[-1] # ベクトルの次元数を取得
            self.vector_store.create_collection(
                multivector_size=emb_dim,
                force_recreate=force_recreate_collection,
            )
            # 画像ベクトルとメタデータをQdrantにバッチ送信
            self.vector_store.store_embeddings(
                embeddings, all_metadata,
                upsert_batch_size=upsert_batch_size, # Qdrantへの書き込みバッチ件数
                wait=upsert_wait,
            )
            logger.info("All embeddings stored in Qdrant successfully")

    def search_documents(self, query: str, top_k: int = 3) -> List[Dict[str, Any]]:
        """
        テキストクエリをベクトルに変換して Qdrantで類似検索し、検索結果を返す関数
        """
        if self.retrieval_model is None or self.retrieval_processor is None:
            raise ValueError("Retrieval model not loaded. Call load_retrieval() first.")
        # テキストクエリのベクトル変換
        with torch.no_grad():
            # クエリ文字列を前処理して、モデル入力形式に変換（トークナイズ）
            proc = self.retrieval_processor.process_queries([query]).to(self.retrieval_model.device)
            # 検索用埋め込みベクトルを生成
            q_emb = self.retrieval_model(**proc)
        # Qdrantに埋め込みを渡して類似検索を実行
        hits = self.vector_store.search(q_emb[0], top_k=top_k)
        results = []
        # Qdrantの検索結果を整形
        for h in hits:
            meta = h["payload"]
            page_idx = int(meta.get("global_page_num", h["id"]))
            results.append({
                "pdf_name": meta["pdf_name"],
                "pdf_path": meta["pdf_path"],
                "page_num": meta["page_num"],  # PDF内のページ番号（1始まり）
                "global_page_num": meta["global_page_num"],  # 全体ページ番号（通し番号）
                "score": h["score"],  # 類似度スコア
                "metadata": meta,  # メタ情報
                "image": self.all_document_images[page_idx] if 0 <= page_idx < len(self.all_document_images) else None, #画像
            })
        return results

    def generate_answer(
        self, question: str, search_results: List[Dict[str, Any]], max_new_tokens: int = 500
    ) -> str:
        """
        検索結果から画像を抽出して、ユーザーの質問と合わせてVLMに入力して回答を生成する
        """
        if self.vlm_model is None or self.vlm_processor is None:
            raise ValueError("VLM model not loaded. Call load_vlm() first.")

        # 検索結果から画像を抽出
        imgs = [r["image"] for r in search_results if r.get("image") is not None]

        # VLMに渡すチャット形式の入力を作成
        chat_msgs = [{
            "role": "user",
            "content": [
                # 検索結果の画像を順番に追加
                *[{"type": "image", "image": img} for img in imgs],
                # ユーザー質問をテキストとして追加
                {"type": "text", "text": f"以下の画像を参照して質問に答えてください：\n\n質問: {question}\n\n回答:"}
            ]
        }]

        # モデルが解釈できるチャットテンプレートに整形
        text = self.vlm_processor.apply_chat_template(
            chat_msgs, tokenize=False, add_generation_prompt=True
        )
        # チャットメッセージから画像入力だけを抽出
        img_inputs, _ = process_vision_info(chat_msgs)

        # プロセッサでテキストをトークナイズして、画像を前処理してテンソル化
        inputs = self.vlm_processor(
            text=[text],
            images=img_inputs,
            padding=True,
            return_tensors="pt",
        ).to(self.vlm_model.device)

        # VLMで回答生成
        with torch.no_grad():
            gen_ids = self.vlm_model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
            )

        # プロンプトを除外し、出力部分のみテキストに変換して表示
        trimmed = [out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, gen_ids)]
        decoded = self.vlm_processor.batch_decode(
            trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )
        return decoded[0].strip()

class AdvancedQASystem:
    def __init__(self, rag: MultiModalRAG):
        self.rag = rag

    def answer_question(self, question: str, top_k: int = 3, max_new_tokens: int = 500) -> Dict[str, Any]:
        """
        ユーザーの質問に対して、検索と回答生成をまとめて実行する。
        
        Args:
            question (str): ユーザーの質問（自然文）
            top_k (int): 検索で取得する上位候補数
            max_new_tokens (int): 回答生成の最大トークン数
        """
        # 関連文書を検索
        results = self.rag.search_documents(question, top_k=top_k)
        
        # 検索結果を基に回答を生成
        answer = self.rag.generate_answer(question, results, max_new_tokens=max_new_tokens)

        # 質問文・回答文・参照したソース情報（PDF名・ページ番号・スコア）を辞書で返す
        return {
            "question": question,
            "answer": answer,
            "sources": [
                {"pdf_name": r["pdf_name"], "page_num": r["page_num"], "score": r["score"]}
                for r in results
            ],
        }

# ----------エントリーポイント----------
if __name__ == "__main__":
    setup_logging(level=logging.INFO, to_stdout=True, include_name=True)  
    log_env()

    # MultiModalRAG のインスタンスを初期化
    rag = MultiModalRAG(
        use_qdrant=True,
        qdrant_url="http://qdrant:6333",
        qdrant_api_key="colqwen25-qdrant-api-key",
        collection_name="multi_pdf_documents",
        pdf_dpi=150, # PDF→画像変換時の解像度（DPI）
        pdf_fmt="jpeg",
        use_pdftocairo=True,
    )

    # 検索用モデルのみロード
    rag.load_retrieval()

    # サンプルPDFファイルを指定
    pdf_paths = ["sample1.pdf", "sample2.pdf"]
    
    existing_pdfs = [p for p in pdf_paths if os.path.exists(p)]
    # 実際に存在するPDFだけを対象にする
    if not existing_pdfs:
        logger.warning("No input PDFs found. Put your PDFs next to this script.")
    else:
        # PDF群 → 画像変換 → 埋め込み生成 → Qdrantに保存
        rag.process_multiple_pdfs(
            existing_pdfs,
            batch_size=16,           # DataLoaderのバッチサイズ
            num_workers=8,            # DataLoaderの並列ワーカー数（CPUリソースに依存）
            prefetch_factor=2,        # ワーカーごとの先読みバッチ数
            force_recreate_collection=True, # 既存コレクションを削除して再作成
            upsert_batch_size=256,    # Qdrantに送信するバッチサイズ
            upsert_wait=False,        # Trueならアップサート完了を待つ
        )

        # 回答生成（VLM）が必要になった段階でロード
        rag.load_vlm()

        # QAシステムをラップするクラスを用意
        qa = AdvancedQASystem(rag)

        logger.info("==== 前処理完了 ====")

[2025-08-24 09:41:19,691] INFO: __main__: CUDA available: True
[2025-08-24 09:41:19,729] INFO: __main__: GPU name: NVIDIA A100 80GB PCIe
[2025-08-24 09:41:19,732] INFO: __main__: FlashAttention2 available: True


/tmp/ipykernel_2363/1425495769.py:57: UserWarning: Api key is used with an insecure connection.
  self.client = QdrantClient(


[2025-08-24 09:41:20,071] INFO: __main__: Qdrant client initialized with URL: http://qdrant:6333
[2025-08-24 09:41:20,073] INFO: __main__: Loading retrieval model: vidore/colqwen2.5-v0.2


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

[2025-08-24 09:42:42,373] INFO: __main__: throughput: 10.76 images/sec, total=282
[2025-08-24 09:42:42,374] INFO: __main__: Created embeddings for 282 pages from 2 PDFs
[2025-08-24 09:42:42,384] INFO: __main__: Deleting existing collection: multi_pdf_documents
[2025-08-24 09:42:42,548] INFO: __main__: Created collection multi_pdf_documents with multivector support


[2025-08-24 09:42:48,532] INFO: __main__: Qdrant upsert 256 pts in 3.72s
[2025-08-24 09:42:49,281] INFO: __main__: Qdrant upsert 26 pts in 0.33s
[2025-08-24 09:42:49,282] INFO: __main__: Stored total 282 embeddings in Qdrant
[2025-08-24 09:42:49,319] INFO: __main__: All embeddings stored in Qdrant successfully
[2025-08-24 09:42:49,496] INFO: __main__: Loading VLM model: Qwen/Qwen2.5-VL-32B-Instruct


Loading checkpoint shards:   0%|          | 0/18 [00:00<?, ?it/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


[2025-08-24 09:43:06,875] INFO: __main__: VLM loaded in 17.38s
[2025-08-24 09:43:06,877] INFO: __main__: ==== 前処理完了 ====


In [3]:
queries = [
    "液晶モニター文書の主な内容は何ですか？",
    "メニューの図において、明るさ、コントラスト、画質のそれぞれの数値を教えて下さい。",
    "モニターの角度について、何度に調整できますか？",
    "MODE設定の画面において、ムービーのアイコンは左から何番目にある？",
    "動作温度は？",
    "スタンドをモニターに接続する方法は？",
    "モニター背面の接続端子の種類を教えて下さい。",
    "解像度の最小値と最大値は？",
    "スタンドを含めたモニタ全体の寸法は？",
    "Kensingtonセキュリティケーブルはどのようにして固定していますか？",        
    "令和７年情報通信白書の主な内容は何ですか？",
    "2014年から2024年にかけてLINE利用率が最も成長した年代は？",
    "2020年以降、2番目に高い利用率の動画サービスは？",
    "ニュース目的のソーシャルメディア利用率において、全年代でトップのメディアは何？",
    "海外プラットフォーム事業者の売上高で最も成長性しているのは？",
    "日本における生成AI導入に際しての懸念事項で最も多いものは？",
    "生成AI関連市場の市場構成図について解説してください。",
    "DX・イノベーション加速プラン2030はどのような図を表していますか？",
    "テレワークの導入形態においてサテライトオフィス勤務は何%になっていますか？"
    "AIによるリスク例の体系的な分類案において、経済活動に関するリスクを挙げてください。",
    "米国の民間情報化投資は日本と比較してどのように遷移していますか？",
    "インターネット利用時に不安を感じないひとの割合は？",
    "グローバルICT市場の国・地域別シェアの推計（カテゴリー別）（2023年）において、クラウドの米国シェアは？",
    "海底ケーブルの使用帯域幅の推移において、コンテンツプロバイダーはどのように遷移していますか？",
    "日本のデータセンターサービス市場規模（売上高）の推移予測において、前年比成長率はどのように遷移していますか？",
    "世界のICT市場における時価総額上位15社の変遷において、2025年に向上した企業を教えてもらえますか？",
    "我が国の周波数帯ごとの主な用途と電波の特徴の図は何を表していますか？",
    "衛星・HAPSによる通信サービスの提供イメージの図は何を表していますか？",
    "日本のデータセンターの設置状況（地域別サーバールーム面積）において100,000㎡以上の地域はどこですか？"
]

import time
from IPython.display import display, clear_output

print(f"=== 開始: {len(queries)}件の質問を処理します ===\n")

for i, q in enumerate(queries, 1):
    print("="*80)
    print(f"【{i}/{len(queries)}】質問: {q}")
    print("-"*80)
    
    start_time = time.time()
    
    res = qa.answer_question(q, top_k=5, max_new_tokens=300)
    
    elapsed_time = time.time() - start_time
    
    print("【回答】")
    print(res["answer"])
    print(f"\n【参照ソース】")
    for src in res["sources"]:
        print(f"- {src['pdf_name']} ページ{src['page_num']} (スコア:{src['score']:.3f})")
    
    print(f"\n処理時間: {elapsed_time:.2f}秒")
    print(f"進捗: {i}/{len(queries)} 完了 ({100*i/len(queries):.1f}%)")
    print()

print(" 全ての質問の処理が完了しました")

=== 開始: 28件の質問を処理します ===

【1/28】質問: 液晶モニター文書の主な内容は何ですか？
--------------------------------------------------------------------------------
[2025-08-24 09:43:07,034] INFO: __main__: Found 5 results
【回答】
### 回答:
液晶モニター文書の主な内容は、以下のとおりです：

1. **製品情報と説明**:
   - 文書はLG製液晶モニター（モデル: IPS225V, IPS235V）に関する取扱説明書であり、購入者に対して感謝の意を表しています。
   - 使用前に本取扱説明書をよく読んで保管することを推奨しています。

2. **目次と構成**:
   - 文書は以下の主要なセクションで構成されています：
     - **組み立てと準備**: 同梱品、各部の名称、OSDメニューの説明、モニターの設置方法（スタンドの取り付け/取り外し、テーブルへの設置、壁への取り付け）。
     - **モニターの接続**: PCへの接続方法（D-SUB接続、DVI-D接続、HDMI接続）。
     - **設定の方法**: メインメニューへのアクセス、メニュー項目の変更方法、画質調整、モード設定、デュアルパッケージ設定など。
     - **トラブルシューティング**: 問題解決のための対処方法。
     - **仕様**: 製品の詳細仕様（

【参照ソース】
- sample1.pdf ページ1 (スコア:18.606)
- sample1.pdf ページ2 (スコア:15.781)
- sample2.pdf ページ4 (スコア:14.689)
- sample1.pdf ページ12 (スコア:14.242)
- sample1.pdf ページ4 (スコア:14.205)

処理時間: 23.54秒
進捗: 1/28 完了 (3.6%)

【2/28】質問: メニューの図において、明るさ、コントラスト、画質のそれぞれの数値を教えて下さい。
---------------------------------------------------------------------